In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_4356/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import numpy as np
from keras.datasets import fashion_mnist
import keras
import matplotlib.pyplot as plt

fashion_mnist=keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


# # Shuffle train_images and train_labels using the same random permutation
# shuffle = np.random.permutation(60000)
# train_images = train_images[shuffle]
# train_labels = train_labels[shuffle]

2024-02-21 12:21:44.460884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 12:21:44.460933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 12:21:44.461948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 12:21:44.470432: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 12:21:45.343126: W tensorflow/compiler/tf2

In [3]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return x > 0

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(s):
    return s * (1 - s)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

In [4]:
class NeuralNetwork:
    def __init__(self, layer_sizes, activation='relu'):
        self.layer_sizes = layer_sizes
        self.activation, self.activation_derivative = self.set_activation_functions(activation)
        self.parameters = self.initialize_parameters(activation)

    def set_activation_functions(self, activation):
        if activation == 'relu':
            return relu, relu_derivative
        elif activation == 'sigmoid':
            return sigmoid, sigmoid_derivative
        elif activation == 'tanh':
            return tanh, tanh_derivative
        else:
            raise ValueError("Unsupported activation function")
    
    def initialize_parameters(self, activation):
        parameters = {}
        for l in range(1, len(self.layer_sizes)):
            if activation == 'relu':
                std_dev = np.sqrt(2. / self.layer_sizes[l-1]) # He Init
            else:
                std_dev = np.sqrt(1. / self.layer_sizes[l-1]) # Xavier Normal
                
            parameters['W' + str(l)] = np.random.randn(self.layer_sizes[l], self.layer_sizes[l-1]) * std_dev
            parameters['b' + str(l)] = np.zeros((self.layer_sizes[l], 1))
        return parameters
    
    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z))
        return expZ / expZ.sum(axis=0, keepdims=True)
    
    def cross_entropy(self, Y, Y_hat):
        m = Y.shape[1]
        loss = -np.sum(Y * np.log(Y_hat + 1e-9))/m
        return loss
    
    def forward_propagation(self, X):
        caches = {}
        H = X
        L = len(self.parameters) // 2
        
        for l in range(1, L):
            H_prev = H
            A = np.dot(self.parameters['W' + str(l)], H_prev) + self.parameters['b' + str(l)]
            H = self.activation(A) 
            caches['A' + str(l)] = A
            caches['H' + str(l)] = H
        
        AL = np.dot(self.parameters['W' + str(L)], H) + self.parameters['b' + str(L)]
        HL = self.softmax(AL)
        caches['A' + str(L)] = AL
        caches['H' + str(L)] = HL
        return HL, caches
    
    def backpropagation(self, X, Y, caches):
        grads = {}
        L = len(self.parameters) // 2 # Number of layers
        m = X.shape[1]
        Y = Y.reshape(caches['H' + str(L)].shape) # Ensure same shape as output layer

        # Initializing backpropagation and Output layer gradient
        dAL = caches['H' + str(L)] - Y
        grads["dW" + str(L)] = 1./m * np.dot(dAL, caches['H' + str(L-1)].T)
        grads["db" + str(L)] = 1./m * np.sum(dAL, axis=1, keepdims=True)

        for l in reversed(range(1, L)):
            dH = np.dot(self.parameters["W" + str(l+1)].T, dAL) # dH_prev
            dA = self.activation_derivative(caches['A' + str(l)]) * dH # Element wise multiplication between 2 vectors
            if l > 1:
                grads["dW" + str(l)] = 1./m * np.dot(dA, caches['H' + str(l-1)].T)
            else: # For the first hidden layer, use X 
                grads["dW" + str(l)] = 1./m * np.dot(dA, X.T)
            grads["db" + str(l)] = 1./m * np.sum(dA, axis=1, keepdims=True)
            dAL = dA  # For the next iteration. Prepare dAL for next layer (if not the first layer)

        return grads
    
    def update_parameters(self, grads, learning_rate):
        L = len(self.parameters) // 2
        for l in range(L):
            self.parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l+1)]
            self.parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l+1)]

    def update_parameters_with_momentum_or_NAG(self, grads, learning_rate, momentum, v):
        L = len(self.parameters) // 2
        
        # Update parameters with momentum
        for l in range(1, L+1):
            v["dW" + str(l)] = momentum * v["dW" + str(l)] + learning_rate * grads["dW" + str(l)]
            v["db" + str(l)] = momentum * v["db" + str(l)] + learning_rate * grads["db" + str(l)]
            
            self.parameters["W" + str(l)] -= v["dW" + str(l)]
            self.parameters["b" + str(l)] -= v["db" + str(l)]

        return v
    
    def update_parameters_for_Adagrad(self,learning_rate, grads, r, l):
        epsilon = 1e-9 # Smoothing term to avoid division by zero
        self.parameters['W' + str(l)] -= learning_rate * grads['dW' + str(l)] / (np.sqrt(r['dW' + str(l)]) + epsilon)
        self.parameters['b' + str(l)] -= learning_rate * grads['db' + str(l)] / (np.sqrt(r['db' + str(l)]) + epsilon)

    def update_parameters_for_RMSprop(self, grads, learning_rate, beta, v_w_and_b):
        L = len(self.parameters) // 2
        epsilon=1e-9
        
        for l in range(1, L+1):
            # compute intermetiate values
            v_w_and_b['dW' + str(l)] = beta * v_w_and_b['dW' + str(l)] + (1 - beta) * np.square(grads['dW' + str(l)])
            v_w_and_b['db' + str(l)] = beta * v_w_and_b['db' + str(l)] + (1 - beta) * np.square(grads['db' + str(l)])
            
            # update parameters
            self.parameters['W' + str(l)] -= learning_rate * grads['dW' + str(l)] / (np.sqrt(v_w_and_b['dW' + str(l)]) + epsilon)
            self.parameters['b' + str(l)] -= learning_rate * grads['db' + str(l)] / (np.sqrt(v_w_and_b['db' + str(l)]) + epsilon)

        return v_w_and_b

    def update_parameters_for_Adam(self, learning_rate, m_w_and_b_hat_dW, v_w_and_b_hat_dW, m_w_and_b_hat_db, v_w_and_b_hat_db, l):
        epsilon=1e-9
        self.parameters['W' + str(l)] -= learning_rate * m_w_and_b_hat_dW / (np.sqrt(v_w_and_b_hat_dW) + epsilon)
        self.parameters['b' + str(l)] -= learning_rate * m_w_and_b_hat_db / (np.sqrt(v_w_and_b_hat_db) + epsilon)

In [5]:
def convert_labels_to_one_hot(labels, classes):
    return np.eye(classes)[labels].T

def preprocess_data(train_images, train_labels, test_images, test_labels):
    X_train = train_images.reshape(train_images.shape[0], -1).T / 255.
    X_test = test_images.reshape(test_images.shape[0], -1).T / 255.
    
    Y_train = convert_labels_to_one_hot(train_labels, 10)
    Y_test = convert_labels_to_one_hot(test_labels, 10)
    
    return X_train, Y_train, X_test, Y_test  

In [6]:
def train_GD(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=10, learning_rate=0.01, activation='relu'):
    np.random.seed(1) 
    nn = NeuralNetwork(layer_architecture, activation)
    m = Y_train.shape[1]
    
    for epoch in range(epochs):
        HL, caches = nn.forward_propagation(X_train)
        loss = nn.cross_entropy(Y_train, HL)
        grads = nn.backpropagation(X_train, Y_train, caches)
        nn.update_parameters(grads, learning_rate)
        
        if epoch % 1 == 0:

            print("Epoch %i, Training loss: %f" % (epoch, loss))
    
            # Evaluate model on whole test data after each epoch
            predictions, _ = nn.forward_propagation(X_test)
            accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
            print(f"Test accuracy : {accuracy}\n")
      

def train_SGD(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=10, activation='relu', learning_rate=0.1, batch_size=1):
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    
    m = X_train.shape[1]  # Number of training examples
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]
            
            HL, caches = nn.forward_propagation(X_batch_train)
            
            # Compute loss for the mini-batch
            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss # Aggregate loss
            
            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)
            nn.update_parameters(grads, learning_rate)
        
        epoch_loss /= m # Average loss over all mini-batches
        
        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))
        
        # Evaluate model on whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy : {accuracy}\n")

def train_SGD_with_Momentum(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.1, momentum=0.9, batch_size=1):
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    m = X_train.shape[1]

    v = {} # history vector for each parameter
    for l in range(1, len(nn.layer_sizes)):
        v['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        v['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])
    
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]
            
            HL, caches = nn.forward_propagation(X_batch_train)
            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss
            
            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)
        
            # Update parameters with momentum
            nn.update_parameters_with_momentum(grads, learning_rate, momentum, v)
        
        epoch_loss /= m  # Average loss over all mini-batches
        
        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))
        
        # Evaluate model on the whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy: {accuracy}\n")


def train_SGD_with_NAG(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.1, momentum=0.9, batch_size=1):
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    m = X_train.shape[1]

    v = {}  # Initialize velocity for NAG
    for l in range(1, len(nn.layer_sizes)):
        v['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        v['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])

    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]

            # Look-ahead step
            v_w_and_v_b = {}
            for l in range(1, len(nn.layer_sizes)):
                v_w_and_v_b["W" + str(l)] = nn.parameters["W" + str(l)] - momentum * v["dW" + str(l)]
                v_w_and_v_b["b" + str(l)] = nn.parameters["b" + str(l)] - momentum * v["db" + str(l)]
            
            # Use v_w_and_v_b parameters for forward propagation
            nn.parameters, original_parameters = v_w_and_v_b, nn.parameters
            HL, caches = nn.forward_propagation(X_batch_train)
            nn.parameters = original_parameters  # Restore original parameters

            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss

            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)

            # Update parameters with computed gradients and NAG
            v = nn.update_parameters_with_momentum_or_NAG(grads, learning_rate, momentum, v)


        epoch_loss /= m  # Average loss over all mini-batches

        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))

        # Evaluate model on the whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy: {accuracy}\n")

def train_SGD_with_Adagrad(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.1, batch_size=1):
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    m = X_train.shape[1]

    r = {}  # squared gradients summation for Adagrad
    for l in range(1, len(nn.layer_sizes)):
        r['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        r['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])

    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]

            HL, caches = nn.forward_propagation(X_batch_train)
            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss

            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)

            # Update parameters with Adagrad
            for l in range(1, len(nn.layer_sizes)):
                r['dW' + str(l)] += np.square(grads['dW' + str(l)])
                r['db' + str(l)] += np.square(grads['db' + str(l)])
                nn.update_parameters_for_Adagrad(learning_rate, grads, r, l)

        epoch_loss /= m  # Average loss over all mini-batches

        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))

        # Evaluate model on the whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy: {accuracy}\n")

def train_SGD_with_RMSprop(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.01, beta=0.9, batch_size=1):
    # https://iitm-pod.slides.com/arunprakash_ai/cs6910-lecture-52/fullscreen#/0/20/0 
    # Slide 21
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    m = X_train.shape[1]

    v_w_and_b = {}  # squared gradients summation for RMSprop
    for l in range(1, len(nn.layer_sizes)):
        v_w_and_b['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        v_w_and_b['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])

    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]

            HL, caches = nn.forward_propagation(X_batch_train)
            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss

            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)

            # Update parameters with RMSprop
            v_w_and_b = nn.update_parameters_for_RMSprop(grads, learning_rate, beta, v_w_and_b)

        epoch_loss /= m  # Average loss over all mini-batches

        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))

        # Evaluate model on the whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy: {accuracy}\n")

def train_SGD_with_Adam(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.001, beta1=0.9, beta2=0.999,batch_size=1):
    # https://iitm-pod.slides.com/arunprakash_ai/cs6910-lecture-52/fullscreen#/0/41
    # Slide 42
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    m = X_train.shape[1]

    m_w_and_b = {}  # Momentum conservation
    v_w_and_b = {}  # RMSprop - Exp. Wt. Avg.
    for l in range(1, len(nn.layer_sizes)):
        m_w_and_b['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        m_w_and_b['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])
        v_w_and_b['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        v_w_and_b['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])

    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]

            HL, caches = nn.forward_propagation(X_batch_train)
            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss

            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)

            # Update parameters with Adam
            for l in range(1, len(nn.layer_sizes)):
                m_w_and_b['dW' + str(l)] = beta1 * m_w_and_b['dW' + str(l)] + (1 - beta1) * grads['dW' + str(l)]
                m_w_and_b['db' + str(l)] = beta1 * m_w_and_b['db' + str(l)] + (1 - beta1) * grads['db' + str(l)]

                v_w_and_b['dW' + str(l)] = beta2 * v_w_and_b['dW' + str(l)] + (1 - beta2) * np.square(grads['dW' + str(l)])
                v_w_and_b['db' + str(l)] = beta2 * v_w_and_b['db' + str(l)] + (1 - beta2) * np.square(grads['db' + str(l)])

                # Correct the bias in first moment
                m_w_and_b_hat_dW = m_w_and_b['dW' + str(l)] / (1 - beta1 ** (epoch + 1))
                m_w_and_b_hat_db = m_w_and_b['db' + str(l)] / (1 - beta1 ** (epoch + 1))

                # Correct the bias in second moment
                v_w_and_b_hat_dW = v_w_and_b['dW' + str(l)] / (1 - beta2 ** (epoch + 1))
                v_w_and_b_hat_db = v_w_and_b['db' + str(l)] / (1 - beta2 ** (epoch + 1))

                nn.update_parameters_for_Adam(learning_rate, m_w_and_b_hat_dW, v_w_and_b_hat_dW, m_w_and_b_hat_db, v_w_and_b_hat_db, l)

                # nn.parameters['W' + str(l)] -= learning_rate * m_w_and_b_hat_dW / (np.sqrt(v_w_and_b_hat_dW) + epsilon)
                # nn.parameters['b' + str(l)] -= learning_rate * m_w_and_b_hat_db / (np.sqrt(v_w_and_b_hat_db) + epsilon)

        epoch_loss /= m  # Average loss over all mini-batches

        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))

        # Evaluate model on the whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy: {accuracy}\n")

def train_SGD_with_Nadam(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.01, beta1=0.9, beta2=0.999, batch_size=1):
    np.random.seed(1)
    nn = NeuralNetwork(layer_architecture, activation)
    m = X_train.shape[1]

    m_w_and_b = {}  # Momentum
    v_w_and_b = {}  # Velocity
    for l in range(1, len(nn.layer_sizes)):
        m_w_and_b['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        m_w_and_b['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])
        v_w_and_b['dW' + str(l)] = np.zeros_like(nn.parameters['W' + str(l)])
        v_w_and_b['db' + str(l)] = np.zeros_like(nn.parameters['b' + str(l)])

    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, m, batch_size):
            X_batch_train = X_train[:, i:i+batch_size]
            Y_batch_train = Y_train[:, i:i+batch_size]

            HL, caches = nn.forward_propagation(X_batch_train)
            mini_batch_loss = nn.cross_entropy(Y_batch_train, HL)
            epoch_loss += mini_batch_loss

            grads = nn.backpropagation(X_batch_train, Y_batch_train, caches)

            # Update parameters with Nadam
            for l in range(1, len(nn.layer_sizes)):
                # computer intermediate values
                m_w_and_b_dW = beta1 * m_w_and_b['dW' + str(l)] + (1 - beta1) * grads['dW' + str(l)]
                m_w_and_b_db = beta1 * m_w_and_b['db' + str(l)] + (1 - beta1) * grads['db' + str(l)]

                v_w_and_b_dW = beta2 * v_w_and_b['dW' + str(l)] + (1 - beta2) * np.square(grads['dW' + str(l)])
                v_w_and_b_db = beta2 * v_w_and_b['db' + str(l)] + (1 - beta2) * np.square(grads['db' + str(l)])

                # Correct the bias
                m_w_and_b_hat_dW = m_w_and_b_dW / (1 - beta1 ** (epoch + 1))
                m_w_and_b_hat_db = m_w_and_b_db / (1 - beta1 ** (epoch + 1))
                v_w_and_b_hat_dW = v_w_and_b_dW / (1 - beta2 ** (epoch + 1))
                v_w_and_b_hat_db = v_w_and_b_db / (1 - beta2 ** (epoch + 1))

                # Nesterov update
                # nesterov_dW = beta1 * m_w_and_b_hat_dW + ((1 - beta1) * grads['dW' + str(l)]) / (1 - beta1 ** (epoch + 1))
                # nesterov_db = beta1 * m_w_and_b_hat_db + ((1 - beta1) * grads['db' + str(l)]) / (1 - beta1 ** (epoch + 1))

                # update parameters
                nn.parameters['W' + str(l)] -= learning_rate * (beta1 * m_w_and_b_hat_dW + ((1 - beta1) * grads['dW' + str(l)]) / (1 - beta1 ** (epoch + 1))) / (np.sqrt(v_w_and_b_hat_dW) + 1e-9)
                nn.parameters['b' + str(l)] -= learning_rate * (m_w_and_b_hat_db + ((1 - beta1) * grads['db' + str(l)]) / (1 - beta1 ** (epoch + 1))) / (np.sqrt(v_w_and_b_hat_db) + 1e-9)

                # Update moving averages
                m_w_and_b['dW' + str(l)] = m_w_and_b_dW
                m_w_and_b['db' + str(l)] = m_w_and_b_db
                v_w_and_b['dW' + str(l)] = v_w_and_b_dW
                v_w_and_b['db' + str(l)] = v_w_and_b_db

        epoch_loss /= m  # Average loss over all mini-batches

        print("Epoch %i, Training loss: %f" % (epoch, epoch_loss))

        # Evaluate model on the whole test data after each epoch
        predictions, _ = nn.forward_propagation(X_test)
        accuracy = np.mean(np.argmax(predictions, axis=0) == np.argmax(Y_test, axis=0))
        print(f"Test accuracy: {accuracy}\n")


X_train, Y_train, X_test, Y_test = preprocess_data(train_images, train_labels, test_images, test_labels)
layer_architecture = [X_train.shape[0], 32, 64, 10]
# train_GD(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=30, activation='relu', learning_rate=0.1)
# train_SGD(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=30, activation='relu', learning_rate=0.001, batch_size=16) 
# train_SGD_with_Momentum(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=30, activation='tanh', learning_rate=0.01, momentum=0.5, batch_size=1)
# train_SGD_with_NAG(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=30, activation='relu', learning_rate=0.01, momentum=0.5, batch_size=1)
# train_SGD_with_Adagrad(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=30, activation='relu', learning_rate=0.01, batch_size=1)
train_SGD_with_RMSprop(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.01, beta=0.5, batch_size=1)
# train_SGD_with_Adam(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.001, beta1=0.9, beta2=0.999, batch_size=1)
# train_SGD_with_Nadam(layer_architecture, X_train, Y_train, X_test, Y_test, epochs=3, activation='tanh', learning_rate=0.01, beta1=0.9, beta2=0.999, batch_size=1)




Epoch 0, Training loss: 1.364681
Test accuracy: 0.7481



/tmp/ipykernel_4356/2855403050.py:31: RuntimeWarning: invalid value encountered in divide
  return expZ / expZ.sum(axis=0, keepdims=True)


Epoch 1, Training loss: 1.401335
Test accuracy: 0.6346



KeyboardInterrupt: 